In [ ]:

# -*- coding: utf-8 -*-

# Installation des dépendances en une seule fois
!pip install datasets Pillow torch torchvision transformers scikit-learn joblib

from huggingface_hub import notebook_login, hf_hub_download
import os
import tarfile
import numpy as np
import torch
import joblib
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import roc_auc_score
from transformers import AutoImageProcessor, EfficientFormerModel, AutoModelForImageClassification, AutoFeatureExtractor

# Connexion Hugging Face
notebook_login()

# -------------------------------
# Téléchargement et extraction des données
# -------------------------------
repo_id = "Team198523/dataset_image"
filename = "VisA_20220922.tar"
extract_path = "VisA_20220922_extracted"

# Télécharger et extraire l'archive
if not os.path.exists(extract_path):
    tar_path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset", revision="main")
    os.makedirs(extract_path, exist_ok=True)
    with tarfile.open(tar_path, "r:*") as tar:
        tar.extractall(path=extract_path)

# -------------------------------
# Chargement et prétraitement des images
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_and_preprocess_image(img_path):
    image = Image.open(img_path).convert("RGB")
    return transform(image).unsqueeze(0)

def get_image_paths(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

def build_dataset(base_path):
    image_paths = []
    labels = []
    categories = []

    for category in os.listdir(base_path):
        category_path = os.path.join(base_path, category)
        if not os.path.isdir(category_path):
            continue

        category_images_path = os.path.join(category_path, "Data", "Images")

        # Images normales
        normal_path = os.path.join(category_images_path, "Normal")
        if os.path.exists(normal_path):
            for img in os.listdir(normal_path):
                image_paths.append(os.path.join(normal_path, img))
                labels.append(0)  # Normal
                categories.append(category)

        # Images anomalies
        anomaly_path = os.path.join(category_images_path, "Anomaly")
        if os.path.exists(anomaly_path):
            for img in os.listdir(anomaly_path):
                image_paths.append(os.path.join(anomaly_path, img))
                labels.append(1)  # Anomalie
                categories.append(category)

    return Dataset.from_dict({
        "image_path": image_paths,
        "label": labels,
        "category": categories
    })

# -------------------------------
# Chargement des modèles
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
image_classifier = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50").to(device).eval()

# -------------------------------
# Extraction de caractéristiques
# -------------------------------
def extract_features(image_paths):
    features = []
    with torch.no_grad():
        for img_path in image_paths:
            inputs = feature_extractor(images=Image.open(img_path).convert("RGB"), return_tensors="pt").to(device)
            outputs = image_classifier(**inputs)
            features.append(outputs.logits.cpu().numpy())
    return np.vstack(features)

# -------------------------------
# Détection d'anomalies
# -------------------------------
dataset = build_dataset(extract_path)
dataset = dataset.cast_column("image_path", DatasetsImage())

normal_images = [img for img, label in zip(dataset["image_path"], dataset["label"]) if label == 0]
normal_features = extract_features(normal_images)
np.save("normal_features.npy", normal_features)

pca = PCA(n_components=min(normal_features.shape[0], normal_features.shape[1]) - 1)
features_reduced = pca.fit_transform(normal_features)
anomaly_detector = EllipticEnvelope(contamination=0.01).fit(features_reduced)

joblib.dump(anomaly_detector, "anomaly_detector.pkl")
joblib.dump(pca, "pca_transformer.pkl")

# -------------------------------
# Test sur les images anormales
# -------------------------------
anomaly_images = [img for img, label in zip(dataset["image_path"], dataset["label"]) if label == 1]
test_features = extract_features(anomaly_images)
test_features_reduced = pca.transform(test_features)
anomaly_scores = anomaly_detector.decision_function(test_features_reduced)
threshold = np.percentile(anomaly_scores, 95)
predictions = anomaly_scores < threshold

# Affichage des résultats
for img_path, score, pred in zip(anomaly_images, anomaly_scores, predictions):
    print(f"{img_path}: {'Anomalie' if pred else 'Normale'} (score: {score:.2f})")

# AUROC Score
true_labels = [1] * len(anomaly_images)
auc = roc_auc_score(true_labels, -anomaly_scores)
print(f"AUROC Score: {auc:.4f}")

# Histogramme des scores
plt.figure(figsize=(10, 5))
plt.hist(anomaly_scores, bins=50, alpha=0.75, color='blue', label="Scores d'anomalie")
plt.axvline(threshold, color='red', linestyle='dashed', linewidth=2, label="Seuil")
plt.xlabel("Score d'anomalie")
plt.ylabel("Nombre d'images")
plt.legend()
plt.title("Distribution des scores d'anomalie")
plt.show()

# Sauvegarde des résultats
import pandas as pd
results_df = pd.DataFrame({"image_path": anomaly_images, "anomaly_score": anomaly_scores, "prediction": ["Anomalie" if pred else "Normale" for pred in predictions]})
results_df.to_csv("detection_results.csv", index=False)
print("Rapport des anomalies sauvegardé dans 'detection_results.csv'")
